## update user contributions


In [1]:
## should do this on a monthly basis because 1) this is will quickly use up BULK API's allotted quantity
## and 2) db2 only updates some usage numbers on a yearly or monthly basis

## currently this script takes over 25 minutes to upload to SF

In [2]:
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'Name'
#object_id = 'ContactToolAssociation__c'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')

salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors
Obtained Salesforce access token ...... True


In [3]:
import sys
import pandas as pd
import time
import datetime

## Obtain information from DB2 

In [4]:
# extract the wiki contributions
wiki_query = "select hits, created_by from jos_wiki_pages where state = '1' and scope = 'site';"
wiki_results_df = pd.read_sql_query(wiki_query,nanohub_db)
display(wiki_results_df.head())

,hits,created_by
0,6540,4843
1,44525,8780
2,5509,12642
3,7863,8757
4,44764,8629


In [5]:
# extract paper contributions; need investigate how to get the usage per paper
paper_contribs_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '3' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
papers_df = pd.read_sql_query(paper_contribs_query,nanohub_db)
display(papers_df.head())

,authorid,hits
0,1871.0,13674
1,2862.0,24552
2,8629.0,24552
3,3858.0,24552
4,1986.0,24552


In [6]:
# extract presentation contributions; need to see how to get usage per presentation
presentations_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '1' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
presentation_df = pd.read_sql_query(presentations_query,nanohub_db)
display(presentation_df)

,authorid,hits
0,-16.0,10468
1,3013.0,6991
2,2862.0,21358
3,45455.0,19139
4,-32.0,21475
...,...,...
7202,NaN,0
7203,NaN,0
7204,NaN,0
7205,NaN,0


In [7]:
sql_query = "select authorid, hits from jos_resources left join jos_author_assoc "\
            "on jos_author_assoc.subid = jos_resources.id where type = '7' and access in ('0','3','1') "\
            "and standalone = '1' and published in ('1');"
authors_query = '''
SELECT author.authorid, res.hits FROM nanohub.jos_tool tool
  LEFT JOIN nanohub.jos_resources res ON res.title = tool.title
  LEFT JOIN nanohub.jos_author_assoc author ON author.subid  = res.id
WHERE res.published = 1 AND res.;
'''
tools_df = pd.read_sql_query(sql_query,nanohub_db)

In [8]:
display(presentation_df.head())

,authorid,hits
0,-16.0,10468
1,3013.0,6991
2,2862.0,21358
3,45455.0,19139
4,-32.0,21475


In [9]:
display(tools_df.head())

,authorid,hits
0,-5.0,251605
1,8629.0,50032
2,3482.0,125516
3,2862.0,251605
4,8629.0,251605


In [10]:
#pre-filtering queries using datetime
date_temp = datetime.datetime.today()
year = date_temp.year
month = date_temp.month
period = 14
if month < 10:
    date_str0 = '\''+ str(year)+'-0'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-0'+str(month)+'-01'+'\''
else:
    date_str0 = '\''+ str(year)+'-'+str(month)+'-00'+'\''
    date_str1 = '\''+str(year)+'-'+str(month)+'-01'+'\''
    
display(date_str0)
display(date_str1)

"'2021-10-00'"

"'2021-10-01'"

In [11]:
# getting the users that used the contributions
sql_query = "select * from jos_author_stats WHERE datetime " \
    "between " + date_str0 +" and " + date_str1 + " and period = 14;"

display(sql_query)

sql_query = "SELECT * FROM jos_author_stats " \
    "WHERE `datetime` > " + "(NOW() - INTERVAL 1 MONTH) AND period = 14;"

display(sql_query)

"select * from jos_author_stats WHERE datetime between '2021-10-00' and '2021-10-01' and period = 14;"

'SELECT * FROM jos_author_stats WHERE `datetime` > (NOW() - INTERVAL 1 MONTH) AND period = 14;'

In [12]:
result_users_df = pd.read_sql_query(sql_query, nanohub_db)
display(result_users_df.tail())

,id,authorid,tool_users,andmore_users,total_users,datetime,period
3380,1287950,88194,NaN,5,5,2021-10-00 00:00:00,14
3381,1287953,10712,NaN,6,6,2021-10-00 00:00:00,14
3382,1287956,337740,1.0,0,1,2021-10-00 00:00:00,14
3383,1287959,338337,1.0,0,1,2021-10-00 00:00:00,14
3384,1287962,16514,NaN,0,0,2021-10-00 00:00:00,14


In [13]:
# getting the users that used the contributions
sql_query = "select * from jos_author_stats WHERE period = 14 ORDER BY datetime DESC LIMIT 100;"
display(sql_query)

test_df = pd.read_sql_query(sql_query, nanohub_db)
display(test_df)


'select * from jos_author_stats WHERE period = 14 ORDER BY datetime DESC LIMIT 100;'

,id,authorid,tool_users,andmore_users,total_users,datetime,period
0,1277810,4843,3514,923,4416,2021-10-00 00:00:00,14
1,1277813,28115,2008,0,2008,2021-10-00 00:00:00,14
2,1277816,3482,82013,788473,834868,2021-10-00 00:00:00,14
3,1277819,9736,51234,917383,945985,2021-10-00 00:00:00,14
4,1277822,10617,8577,5608,13235,2021-10-00 00:00:00,14
...,...,...,...,...,...,...,...
95,1278095,83475,161,0,161,2021-10-00 00:00:00,14
96,1278098,21711,2243,1857,3940,2021-10-00 00:00:00,14
97,1278101,46626,674,0,674,2021-10-00 00:00:00,14
98,1278104,123827,143,0,143,2021-10-00 00:00:00,14


In [14]:
# for Tanya
display(result_users_df.loc[result_users_df['authorid'] == 29294])

,id,authorid,tool_users,andmore_users,total_users,datetime,period
271,1278623,29294,4609.0,19282,23639,2021-10-00 00:00:00,14


In [15]:
display(wiki_results_df.head(2))
display(papers_df.head(2))
display(presentation_df.head(2))
display(tools_df.head(2))
display(result_users_df.head(2))
print(result_users_df.shape)

,hits,created_by
0,6540,4843
1,44525,8780


,authorid,hits
0,1871.0,13674
1,2862.0,24552


,authorid,hits
0,-16.0,10468
1,3013.0,6991


,authorid,hits
0,-5.0,251605
1,8629.0,50032


,id,authorid,tool_users,andmore_users,total_users,datetime,period
0,1277810,4843,3514.0,923,4416,2021-10-00 00:00:00,14
1,1277813,28115,2008.0,0,2008,2021-10-00 00:00:00,14


(3385, 7)


In [16]:
display(result_users_df.loc[result_users_df['authorid'] == 29294])

,id,authorid,tool_users,andmore_users,total_users,datetime,period
271,1278623,29294,4609.0,19282,23639,2021-10-00 00:00:00,14


In [17]:
## get rid of all negative authorids from all of the dataframes
wikidrops = []
wikikeeps = []
for ind,val in enumerate(wiki_results_df['created_by'].to_list()):
    if val <= 0:
        wikidrops.append(ind)
    else:
        wikikeeps.append(ind)
wiki_results_df = wiki_results_df.iloc[wikikeeps,:]

In [18]:
display(wiki_results_df.head(2))

,hits,created_by
0,6540,4843
1,44525,8780


In [19]:
## papers df 
pdrops = []
pkeeps = []
for ind,val in enumerate(papers_df['authorid'].to_list()):
    if val <= 0:
        pdrops.append(ind)
    else:
        pkeeps.append(ind)
papers_df = papers_df.iloc[pkeeps,:]

In [20]:
display(papers_df.head(2))

,authorid,hits
0,1871.0,13674
1,2862.0,24552


In [21]:
## presentation_df
prdrops = []
prkeeps = []
for ind,val in enumerate(presentation_df['authorid'].to_list()):
    if val <= 0:
        prdrops.append(ind)
    else:
        prkeeps.append(ind)
presentation_df = presentation_df.iloc[prkeeps,:]

In [22]:
display(presentation_df.head(2))

,authorid,hits
1,3013.0,6991
2,2862.0,21358


## roll up information from wiki_results_df, papers_df, and presentation_df

In [23]:
# wiki_results_df

## get the number of unique wiki contributions
wikic = wiki_results_df.groupby(['created_by']).size().reset_index() #need reset_index to save the authorid/created_by field
wikic.columns = ['nanoHUB_user_ID__c','wiki_contributions__c']
wikic = pd.DataFrame(wikic)

## rollup usage
wikic2 = wiki_results_df.groupby(['created_by']).sum().reset_index()
wikic2.columns = ['nanoHUB_user_ID__c','wiki_usage__c']
wikic2 = pd.DataFrame(wikic2)

wikic = wikic.join(wikic2,lsuffix='1', rsuffix='2')
display(wikic.head(2))

,nanoHUB_user_ID__c1,wiki_contributions__c,nanoHUB_user_ID__c2,wiki_usage__c
0,1852,3,1852,27718
1,2862,1,2862,5280


In [24]:
wikic = wikic.drop(columns=['nanoHUB_user_ID__c2'])
wikic = wikic.rename(columns={"nanoHUB_user_ID__c1": "nanoHUB_user_ID__c"})
display(wikic.head(2))

,nanoHUB_user_ID__c,wiki_contributions__c,wiki_usage__c
0,1852,3,27718
1,2862,1,5280


In [25]:
# papers_df - just need the number of unique contributions

## get the number of unique paper contributions
papersc = papers_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
papersc.columns = ['nanoHUB_user_ID__c','paper_contributions__c']
papersc = pd.DataFrame(papersc)

## to add in the number of paper hits in the future ##TODO

display(papersc.head(2))

,nanoHUB_user_ID__c,paper_contributions__c
0,1852.0,3
1,1871.0,1


In [26]:
# presentation_df - just need the number of unique contributions

## get the number of unique presentation contributions
pres_c = presentation_df.groupby(['authorid']).size().reset_index() #need reset_index to save the authorid/created_by field
pres_c.columns = ['nanoHUB_user_ID__c','presentations_contributions__c']
pres_c = pd.DataFrame(pres_c)

## to add in the number of paper hits in the future ##TODO

display(pres_c.head(2))

,nanoHUB_user_ID__c,presentations_contributions__c
0,1742.0,2
1,1810.0,1


## Big picture users dataframe development

In [27]:
# rename the authorid in result_users_df, then join all of the dataframes
ovr_df = result_users_df.rename(columns={"authorid": "nanoHUB_user_ID__c"})
ovr_df['contributor__c'] = 0

In [28]:
## get rid of all negative nanohub ids
drops = []
keeps = []
for ind,val in enumerate(ovr_df['nanoHUB_user_ID__c'].to_list()):
    if val <= 0:
        drops.append(ind)
    else:
        keeps.append(ind)
ovr_df = ovr_df.iloc[keeps,:]

ovr_df = ovr_df.drop(columns=['datetime','period','id'])

display(ovr_df.head(2))
display(ovr_df.tail(2))

,nanoHUB_user_ID__c,tool_users,andmore_users,total_users,contributor__c
0,4843,3514.0,923,4416,0
1,28115,2008.0,0,2008,0


,nanoHUB_user_ID__c,tool_users,andmore_users,total_users,contributor__c
3383,338337,1.0,0,1,0
3384,16514,NaN,0,0,0


In [29]:
ovr_df = ovr_df.set_index('nanoHUB_user_ID__c').join(wikic.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(papersc.set_index('nanoHUB_user_ID__c')) 
ovr_df = ovr_df.join(pres_c.set_index('nanoHUB_user_ID__c')) 

display(ovr_df.head(10))
#use .reset_index() to extract it later

,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
nanoHUB_user_ID__c,,,,,,,,
4843,3514.0,923,4416,0,1.0,6540.0,NaN,NaN
28115,2008.0,0,2008,0,NaN,NaN,NaN,NaN
3482,82013.0,788473,834868,0,20.0,259089.0,9.0,133.0
9736,51234.0,917383,945985,0,6.0,40012.0,4.0,39.0
10617,8577.0,5608,13235,0,NaN,NaN,NaN,4.0
14002,51333.0,16057,65121,0,NaN,NaN,1.0,2.0
28836,9985.0,11154,20297,0,NaN,NaN,1.0,8.0
13570,3578.0,4009,7420,0,NaN,NaN,NaN,1.0
10676,19092.0,3038,21497,0,NaN,NaN,NaN,NaN


In [30]:
ovr_df = ovr_df.reset_index()
display(ovr_df.head(10))

,nanoHUB_user_ID__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
0,4843,3514.0,923,4416,0,1.0,6540.0,NaN,NaN
1,28115,2008.0,0,2008,0,NaN,NaN,NaN,NaN
2,3482,82013.0,788473,834868,0,20.0,259089.0,9.0,133.0
3,9736,51234.0,917383,945985,0,6.0,40012.0,4.0,39.0
4,10617,8577.0,5608,13235,0,NaN,NaN,NaN,4.0
5,14002,51333.0,16057,65121,0,NaN,NaN,1.0,2.0
6,28836,9985.0,11154,20297,0,NaN,NaN,1.0,8.0
7,13570,3578.0,4009,7420,0,NaN,NaN,NaN,1.0
8,10676,19092.0,3038,21497,0,NaN,NaN,NaN,NaN
9,37033,12179.0,0,12179,0,NaN,NaN,NaN,NaN


In [31]:
display(len(ovr_df))

2626

## Get the current nanohub user IDs and their tool counts from SF
then begin the dataframe merge and post-processing

In [32]:
# create DB2 to Salesforce API object
db_s = salesforce

In [33]:
# get nanohub ID for contacts to being matching and processing
sf_username_df = db_s.query_data('SELECT nanoHUB_user_ID__c,Total_tool_contributions__c FROM Contact')

# display
display(sf_username_df.head(2))
print(sf_username_df.shape)

[Success] Bulk job creation successful. Job ID = 7505w00000b2dDVAAY
{"id":"7505w00000b2dDVAAY","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-28T03:51:27.000+0000","systemModstamp":"2021-10-28T03:51:27.000+0000","state":"UploadComplete","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","retries":0,"totalProcessingTime":0}
{"id":"7505w00000b2dDVAAY","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-28T03:51:27.000+0000","systemModstamp":"2021-10-28T03:51:27.000+0000","state":"InProgress","concurrencyMode":"Parallel","contentType":"CSV","apiVersion":47.0,"jobType":"V2Query","lineEnding":"LF","columnDelimiter":"COMMA","numberRecordsProcessed":15032,"retries":0,"totalProcessingTime":865}
{"id":"7505w00000b2dDVAAY","operation":"query","object":"Contact","createdById":"0055w00000DM5bOAAT","createdDate":"2021-10-28T03

,Total_tool_contributions__c,nanoHUB_user_ID__c
0,0.0,998.0
1,0.0,1683.0


(247742, 2)


In [34]:
# join the two dataframes
nover_df = sf_username_df.set_index('nanoHUB_user_ID__c').join(ovr_df.set_index('nanoHUB_user_ID__c'))

In [35]:
display(nover_df.head(2))

,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
nanoHUB_user_ID__c,,,,,,,,,
998.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1683.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [36]:
## calculate the non tool contributions
nover_df['And_More_Contributions__c'] = nover_df['wiki_contributions__c']+nover_df['paper_contributions__c']\
        +nover_df['presentations_contributions__c']

In [37]:
# recover nanohub ids 
nover_df = nover_df.reset_index()
display(nover_df.head(2))

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
## remove NAN rows for index
nover_df = nover_df.dropna(subset=['nanoHUB_user_ID__c'])
nover_df.shape

(247724, 11)

In [39]:
## then make the indexes integers
import numpy as np
nover_df['nanoHUB_user_ID__c'] = nover_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
display(nover_df.head(2))

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [40]:
# calculate contributor flag
for index,val in enumerate(nover_df['contributor__c'].to_list()):
    # if any of the entries are non zero/non-NaN, then contributor__c = 1
    if nover_df['total_users'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['wiki_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['paper_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['presentations_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
    elif nover_df['Total_tool_contributions__c'][index] > 0:
        nover_df['contributor__c'][index] = 1
        
display(nover_df.head(10))

[WARNING] [warnings - py.warnings]: /tmp/ipykernel_1150/2408927924.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nover_df['contributor__c'][index] = 1
 [warnings._showwarnmsg:109]
[WARNING] [warnings - py.warnings]: /tmp/ipykernel_1150/2408927924.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nover_df['contributor__c'][index] = 1
 [warnings._showwarnmsg:109]
[WARNING] [warnings - py.warnings]: /tmp/ipykernel_1150/2408927924.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
0,998,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1683,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1684,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1685,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1686,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,1687,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,1688,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1689,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,1690,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,1691,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
display(nover_df.loc[nover_df['nanoHUB_user_ID__c'] == 29294])

,nanoHUB_user_ID__c,Total_tool_contributions__c,tool_users,andmore_users,total_users,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c,And_More_Contributions__c
25508,29294,5.0,4609.0,19282.0,23639.0,1.0,6.0,0.0,NaN,5.0,NaN


## Fitting data to SF format

In [42]:
## creating and populating the sf dataframe
df_sf = pd.DataFrame()
temp_df = nover_df.fillna(0)

# populating df_sf and make integers
df_sf['nanoHUB_user_ID__c'] = temp_df['nanoHUB_user_ID__c'].apply(lambda x: int(x))
df_sf['And_More_Contributions__c'] = temp_df['And_More_Contributions__c'].apply(lambda x: int(x))
df_sf['contributed_tool_usage__c'] = temp_df['tool_users'].apply(lambda x: int(x))
df_sf['contributed_andmore_usage__c'] = temp_df['andmore_users'].apply(lambda x: int(x))
df_sf['contributor__c'] = temp_df['contributor__c']

df_sf['wiki_contributions__c'] = temp_df['wiki_contributions__c']
df_sf['wiki_usage__c'] = temp_df['wiki_usage__c']
df_sf['paper_contributions__c'] = temp_df['paper_contributions__c']
df_sf['presentations_contributions__c'] = temp_df['presentations_contributions__c']

display(df_sf.head(2))

,nanoHUB_user_ID__c,And_More_Contributions__c,contributed_tool_usage__c,contributed_andmore_usage__c,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
0,998,0,0,0,0.0,0.0,0.0,0.0,0.0
1,1683,0,0,0,0.0,0.0,0.0,0.0,0.0


In [43]:
display(df_sf.loc[df_sf['nanoHUB_user_ID__c'] == 29294])

,nanoHUB_user_ID__c,And_More_Contributions__c,contributed_tool_usage__c,contributed_andmore_usage__c,contributor__c,wiki_contributions__c,wiki_usage__c,paper_contributions__c,presentations_contributions__c
25508,29294,0,4609,19282,1.0,6.0,0.0,0.0,5.0


## send data to SF

In [44]:
# create DB2 to Salesforce API object
db_s = salesforce

# specify Salesforce object ID and external ID
db_s.object_id = 'Contact'
db_s.external_id = 'nanoHUB_user_ID__c'

In [45]:
# send data to Salesforce
db_s.send_data(df_sf)

[Success] Bulk job creation successful. Job ID = 7505w00000b2dDaAAI
hello
[Success] CSV upload successful. Job ID = 7505w00000b2dDaAAI
[Success] Closing job successful. Job ID = 7505w00000b2dDaAAI


In [46]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_status())

{'apexProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apiVersion': 47.0,
 'columnDelimiter': 'COMMA',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-28T03:52:00.000+0000',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'id': '7505w00000b2dDaAAI',
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'numberRecordsFailed': 0,
 'numberRecordsProcessed': 0,
 'object': 'Contact',
 'operation': 'upsert',
 'retries': 0,
 'state': 'UploadComplete',
 'systemModstamp': '2021-10-28T03:52:13.000+0000',
 'totalProcessingTime': 0}


In [47]:
pprint(db_s.check_bulk_failed_results())

''
